In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [26]:
folder = 'LiDAR'
files = sorted([f for f in os.listdir(folder) if f.startswith('lidar_step_')],
               key=lambda x: int(x.split('_')[-1].split('.')[0]))

point_clouds = []
target_points = 1024  # You can change this if needed

for f in files:
    path = os.path.join(folder, f)
    points = np.loadtxt(path, delimiter=',')

    # Trim or pad each point cloud to have `target_points` rows
    if points.shape[0] > target_points:
        points = points[:target_points]
    elif points.shape[0] < target_points:
        pad_len = target_points - points.shape[0]
        pad = np.zeros((pad_len, points.shape[1]))
        points = np.vstack((points, pad))

    point_clouds.append(points)

point_clouds = np.array(point_clouds)
print("Shape:", point_clouds.shape)  # Expected: (60, 1024, 3)

Shape: (59, 1024, 3)


In [27]:
print("Files found:", len(files))

Files found: 59


## LiDAR Autoencoder (PointNet-Style)

In [28]:
# Input shape: (1024, 3)
input_points = tf.keras.Input(shape=(1024, 3))

# ----- Encoder -----
x = layers.Conv1D(64, 1, activation='relu')(input_points)
x = layers.Conv1D(128, 1, activation='relu')(x)
x = layers.Conv1D(256, 1, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)  # shape: (None, 256)

# Latent vector
latent = layers.Dense(128, activation='relu')(x)
latent = layers.Dense(64, activation='relu')(latent)

# ----- Decoder -----
x = layers.Dense(256, activation='relu')(latent)
x = layers.Dense(1024 * 3)(x)
output_points = layers.Reshape((1024, 3))(x)

# Build model
autoencoder = tf.keras.Model(inputs=input_points, outputs=output_points)
autoencoder.compile(optimizer='adam', loss='mse')

# Summary
autoencoder.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1024, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 1024, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 1024, 128)      │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 1024, 256)      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_3          │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3072)           │       789,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 1024, 3)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 888,896 (3.39 MB)

 Trainable params: 888,896 (3.39 MB)

 Non-trainable params: 0 (0.00 B)

## Normalize the Data

In [29]:
# Normalize each point cloud to zero mean and unit variance
point_clouds -= np.mean(point_clouds, axis=1, keepdims=True)
point_clouds /= np.std(point_clouds, axis=1, keepdims=True)

## Training Autoencoder

In [30]:
X_train, X_test = train_test_split(point_clouds, test_size=0.1, random_state=42)

# Train
autoencoder.fit(X_train, X_train, 
                validation_data=(X_test, X_test),
                epochs=50, 
                batch_size=8)


Epoch 1/50


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.9937 - val_loss: 0.9526
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.9116 - val_loss: 0.9162
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.8605 - val_loss: 0.8843
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.8589 - val_loss: 0.8754
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.8572 - val_loss: 0.8631
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.8470 - val_loss: 0.8610
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.8376 - val_loss: 0.8523
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.8469 - val_loss: 0.8562
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.8141 - val_loss: 0.8445
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.8224 - val_loss: 0.8457
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.8212 - val_loss: 0.8451
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.8090 - val_loss: 0.8588
Epoch 13/50
